In [405]:
import min_features, daily_return
import importlib
import pandas as pd
import numpy as np
from sklearn.base import clone
from sklearn.metrics import (
    balanced_accuracy_score,
    accuracy_score,
    f1_score,
    matthews_corrcoef,
    precision_score,
    recall_score,
)
from sklearn.inspection import permutation_importance
import warnings
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
warnings.filterwarnings("ignore", message="y_pred contains classes not in y_true")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import time

importlib.reload(min_features)
importlib.reload(daily_return)

min_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]

if min_feats != 'N':
    df_min = min_features.min_features()
    df_daily, feature_sets = daily_return.pull_daily('QQQ', returns) 

    df_main = pd.merge(df_min, df_daily, how='inner', on='Date')
    df_main = df_main.sort_values(by='Date', ascending=False)

    return_cols = df_main.columns[df_main.columns.str.contains("Return_")].to_list()
    daily_cols = [
        c for c in df_daily.iloc[:, 1:].columns
        if "return" not in c.lower()
    ]
    close_cols = df_min.columns[(df_min.columns.str.contains("close_")) | (df_min.columns.str.contains("post_")) | (df_min.columns.str.contains("overnight_"))].to_list()
    min_cols = (
        df_min
        .loc[:, ~df_min.columns.isin(close_cols)]  # drop close_ columns
        .iloc[:, 1:]                               # drop first column
        .columns
        .to_list()
    )
else:
    df_daily, feature_sets = daily_return.pull_daily('QQQ', returns) 
    return_cols = df_daily.columns[df_daily.columns.str.contains("Return_")].to_list()
    past_return_cols = df_daily.columns[df_daily.columns.str.contains("Past_Ret")].to_list()
    daily_cols = [
        c for c in df_daily.iloc[:, 1:].columns
        if "return" not in c.lower()
    ]
    df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
    new_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
    df_main = df_daily[df_daily['Date'] <= '2026-01-21'].copy()

#top_models = pd.read_csv("top_performers2.csv")
print(f'Available Feature Sets: {feature_sets.keys()}')

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])


In [407]:
df_main

,Date,Close,High,Low,Volume,SMA_10,Close_Rel_Max10,Close_Rel_Min10,SMA_25,Close_Rel_Max25,...,Close_slope10,Close_slope25,Close_slope50,Past_Return_1_sum10,Past_Return_2_sum10,Past_Return_3_sum10,Past_Return_5_sum10,Past_Return_10_sum10,Past_Return_20_sum10,Past_Return_30_sum10
6758,2026-01-21,616.280029,620.419983,607.859985,79837900,0.99,0.98,1.02,1.00,0.98,...,-0.00200,0.00064,0.00048,5.0,4.0,5.0,6.0,7.0,7.0,7.0
6757,2026-01-20,608.059998,615.059998,607.049988,81988900,0.98,0.97,1.00,0.98,0.97,...,-0.00168,0.00075,0.00050,5.0,5.0,6.0,7.0,8.0,6.0,8.0
6756,2026-01-16,621.260010,626.080017,618.880005,61058100,1.00,0.99,1.01,1.00,0.99,...,0.00013,0.00080,0.00049,6.0,6.0,6.0,7.0,9.0,6.0,9.0
6755,2026-01-15,621.780029,630.000000,620.750000,53934900,1.00,0.99,1.02,1.00,0.99,...,0.00117,0.00064,0.00046,6.0,5.0,6.0,7.0,9.0,5.0,10.0
6754,2026-01-14,619.549988,623.450012,614.559998,72598700,1.00,0.98,1.02,1.00,0.98,...,0.00188,0.00052,0.00037,5.0,5.0,6.0,6.0,9.0,4.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1999-03-16,43.867695,44.052457,43.207832,4905800,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,2.0,2.0,0.0,0.0,0.0,0.0
3,1999-03-15,43.498177,43.550966,42.152056,6369000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1999-03-12,42.284012,43.207820,41.940883,8743600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1999-03-11,43.339802,43.696128,42.495177,9688600,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# -----------------------------
# Feature Sets
# -----------------------------
ma_all_cols = feature_sets['ma']
ma_lag = [c for c in ma_all_cols if "lag" in c.lower()]
ma_rel = [c for c in ma_all_cols if "rel_" in c.lower()]
ma_sma = [c for c in ma_all_cols if ("sma_" in c.lower()) and ("lag" not in c.lower())]
ma_num = [c for c in ma_all_cols if ("num" in c.lower()) or ("since" in c.lower())]
rsi_cols = feature_sets['rsi']
macd_cols = feature_sets['macd']
volu_cols = feature_sets['volume']
atr_adx_cols = feature_sets['atr_adx']
vola_cols = feature_sets['volatility']
vix_skew_cols = feature_sets['vix_skew']
experimental_slope_cols = feature_sets['experimental_slope']

sets = [ma_lag, ma_rel, ma_sma, ma_num, rsi_cols + macd_cols, volu_cols, atr_adx_cols + vola_cols, vix_skew_cols, experimental_slope_cols]
set_names = ["ma_lag", "ma_rel", "ma_sma", "ma_num", "rsi_macd", "volu", "atr_adx" + "vola", "vix_skew", "experimental_slope"]

# -----------------------------
# Models
# -----------------------------
models = {
    #"xgboost-4": XGBClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    #"xgboost-6": XGBClassifier(n_estimators=600, random_state=42, n_jobs=-1),
    "xgb_first_pass": XGBClassifier(
    n_estimators=800,
    learning_rate=0.02,
    max_depth=5,
    min_child_weight=10,
    subsample=0.7,
    colsample_bytree=0.7,
    gamma=0.3,
    reg_alpha=0.5,
    reg_lambda=15,
    tree_method="hist",
    random_state=42,
    n_jobs=-1)
}

# -----------------------------
# Run grid (feature sets x horizon x train_years, etc.)
# -----------------------------
returns = [2]#, 10, 30]#[2, 5, 10, 20, 30]
#train_years = [5]#[3, 5, 7] 
days_assessed = 230
test_days = [1]
results= []
results_df = pd.DataFrame()
models = {"xgboost-2": XGBClassifier(n_estimators=200, random_state=42, n_jobs=-1)}

def _compute_dist(y):
    """Distribution stats for y in {0,1}."""
    n = int(len(y))
    n_pos = int((y == 1).sum())
    n_neg = int((y == 0).sum())
    return {
        "test_n": n,
        "test_pos_n": n_pos,
        "test_neg_n": n_neg,
        "test_pos_frac": (n_pos / n) if n else np.nan,
        "test_neg_frac": (n_neg / n) if n else np.nan,
    }

def walkback_runs(
    df,
    feature_cols,
    target_col,
    *,
    date_col="Date",
    train_years=6,
    test_days=5,
    step_days=5,
    runs=20,
    horizon_days=1,        # r (used for purge)
    purge_days=None,       # defaults to horizon_days
    fill_inf=0.0,
):
    """
    Deployment-aligned evaluation:
      - For each run, take a 5-day OOT test window stepping back by 5 days.
      - Train on the prior N years (fixed-length window) ending right before test.
      - Purge 'purge_days' from the end of train to avoid overlap leakage for forward-return labels.
      - Score ONLY on the OOT test window (distribution + metrics).
    Returns: long DataFrame with one row per (feature_set/run/model).
    """
    rows = []

    for k in range(runs):

        dfw = df.sort_values("Date").reset_index(drop=True).copy()

        n = len(dfw)
        train_size = 245 * int(train_years)
        test_size = int(test_days)
        step = int(step_days)
        purge = int(purge_days) if purge_days is not None else 0 #int(horizon_days)
        test_end = n - k * step
        test_start = test_end - test_size

        if test_start < 0:
            break

        train_end = test_start - purge
        train_start = train_end - train_size
        if train_start < 0 or train_end <= train_start:
            break
        
        dates = dfw[date_col].to_numpy() if date_col in dfw.columns else None
        dfpi = dfw[train_start:train_end].copy()

        print(f"PI Train: {dates[train_start]} → {dates[train_end-1]}")

        for pi_year in pi_years:

            for min_feat in min_feats:
                
                #feature_cols = [c for c in feature_cols if c not in new_cols]

                perm_cols = perm_list(
                    df=dfpi,
                    feature_cols=feature_cols,
                    target_col=target_col,
                    date_col="Date",
                    purge_days=r, 
                    fill_inf=0.0,
                    pi_year=pi_year,
                    min_feats=min_feat
                )
                print(sorted(perm_cols))
                #perm_cols += new_cols
                
                # Drop any accidental return cols from features (belt+suspenders)
                safe_feature_cols = [c for c in perm_cols if not c.startswith("Return")]

                # Basic numeric cleaning
                dfw[safe_feature_cols] = dfw[safe_feature_cols].replace([np.inf, -np.inf], fill_inf)

                X_all = dfw[safe_feature_cols].to_numpy()
                #y_all = _to_binary(dfw[target_col].to_numpy())
                y_all = dfw[target_col].to_numpy()

                print(
                    f"Run {k+1}/{runs} | "
                    f"Train: {dates[train_start]} → {dates[train_end-1]} | "
                    f"Test: {dates[test_start]} → {dates[test_end-1]} | "
                    f"Train_n={train_end-train_start} | Test_n={test_end-test_start} | "
                    f"(PI Years: {pi_year} - Feats: {min_feat})"
                )

                X_train = X_all[train_start:train_end]
                y_train = y_all[train_start:train_end]
                X_test  = X_all[test_start:test_end]
                y_test  = y_all[test_start:test_end]

                dist = _compute_dist(y_test)

                for model_name, model in models.items():
                    #start_time = time.time()
                    m = clone(model)
                    m.fit(X_train, y_train)

                    preds = m.predict(X_test)
                    proba = np.nan
                    if hasattr(m, "predict_proba"):
                        proba = float(m.predict_proba(X_test)[0, 1])   # prob(class=1)
                    elif hasattr(m, "decision_function"):
                        s = float(m.decision_function(X_test)[0])
                        proba = float(1.0 / (1.0 + np.exp(-s)))        # squash to (0,1)
                    proba = np.nan if np.isnan(proba) else round(round(proba / 0.05) * 0.05, 2)

                    rows.append({
                        "run": k + 1,
                        "model": model_name,
                        "test_days": test_days,
                        "pred": round(proba,2),
                        "acc": float(accuracy_score(y_test, preds)),
                        **dist,
                        "train_n": int(len(y_train)),
                        "train_start": dates[train_start] if dates is not None else train_start,
                        "train_end": dates[train_end - 1] if dates is not None else train_end - 1,
                        "test_start": dates[test_start] if dates is not None else test_start,
                        "test_end": dates[test_end - 1] if dates is not None else test_end - 1,
                        "train_years": train_years,
                        "n_features": len(safe_feature_cols),
                        "pi_size": pi_year,
                        "min_feats": min_feat
                    })
        
    return pd.DataFrame(rows)

def perm_list(
    df,
    feature_cols,
    target_col,
    *,
    date_col="Date",
    purge_days=None, 
    fill_inf=0.0,
    pi_year=1,
    min_feats=6,
    k=1
):

    dfw = df.sort_values("Date").reset_index(drop=True).copy()

    # Drop any accidental return cols from features (belt+suspenders)
    safe_feature_cols = [c for c in feature_cols if not (c.startswith("Return"))]

    # Basic numeric cleaning
    dfw[safe_feature_cols] = dfw[safe_feature_cols].replace([np.inf, -np.inf], fill_inf)

    """
    test_size = int(test_days)
    step = int(step_days)
    purge = int(purge_days) if purge_days is not None else 0 #int(horizon_days)
    """

    X_train = dfw[safe_feature_cols].to_numpy()
    #y_all = _to_binary(dfw[target_col].to_numpy())
    y_train = dfw[target_col].to_numpy()
    dates = dfw[date_col].to_numpy() if date_col in dfw.columns else None

    """
    test_end = n - k * step
    test_start = test_end - test_size
    train_end = test_start - purge
    train_start = train_end - train_size
    

    X_train = X_all[train_start:train_end]
    y_train = y_all[train_start:train_end]
    """
    
    #N_PI = int(len(X_train) * perc_train)
    N_PI = int(242 * pi_year)
    dates_pi = dates[-N_PI:]
    X_pi = X_train[-N_PI:]
    y_pi = y_train[-N_PI:]

    #print(f"{max(dates_pi)} - {min(dates_pi)}")
    # fit model
    m = clone(model).fit(X_train, y_train)

    # permutation importance on training-only slice
    pi = permutation_importance(
        m,
        X_pi,
        y_pi,
        scoring="neg_log_loss",   # or "accuracy", "neg_log_loss", etc.
        n_repeats=10,
        random_state=42,
        n_jobs=-1,
    )

    # pi.importances_mean aligns to feature_cols order
    pi_df = pd.DataFrame({
        "feature": feature_cols,                 # same order used to build X_train
        "pi_mean": pi.importances_mean,
        "pi_std":  pi.importances_std,
    }).sort_values("pi_mean", ascending=False)

    # keep only features with PI > 0
    pi_cols = pi_df['feature'][pi_df['pi_mean'] > .01].to_list()

    if len(pi_cols) < min_feats:
        pi_cols = (
            pi_df.sort_values("pi_mean", ascending=False)
                .head(min_feats)["feature"]
                .tolist()
        )
    print(f"Ran permutation importance for horizon {purge_days} | Len: {N_PI} | Old: {len(feature_cols)} | New: {len(pi_cols)}")
    
    return pi_cols

for test_day in test_days:

    runs = int(days_assessed / test_day)

    for r in returns:

        if r == 2:
            base_cols = experimental_slope_cols + ma_lag + rsi_cols + macd_cols + volu_cols
            base_cols = ma_lag + ma_rel + ma_sma + ma_num + rsi_cols + macd_cols + volu_cols + atr_adx_cols + vola_cols + vix_skew_cols + experimental_slope_cols + new_cols
            base_cols = new_cols
            ### Best Model Config ###
            train_years = [4]
            pi_years = [2]
            min_feats = [6]
            models = {"xgboost-2": XGBClassifier(n_estimators=200, random_state=42, n_jobs=-1)}

        elif r == 5:
            base_cols = experimental_slope_cols + ma_lag + ma_num + rsi_cols + macd_cols + volu_cols
            base_cols = ma_lag + ma_rel + ma_sma + ma_num + rsi_cols + macd_cols + volu_cols + atr_adx_cols + vola_cols + vix_skew_cols + experimental_slope_cols
            list_name = "initial+atradx" #worse
            list_name = "initial+sma" #worse
            list_name = "initial+vixskew" #much worse
            #train_years = 5
            #cols = atr_adx_cols + vola_cols + experimental_slope_cols + ma_lag + ma_num + ma_rel + ma_sma + rsi_cols + macd_cols + volu_cols + vix_skew_cols
        elif r == 10:
            base_cols = atr_adx_cols + vola_cols + ma_num + volu_cols + ma_sma
            base_cols = ma_lag + ma_rel + ma_sma + ma_num + rsi_cols + macd_cols + volu_cols + atr_adx_cols + vola_cols + vix_skew_cols + experimental_slope_cols
            list_name = "initial+sma" # better
            list_name = "initial+sma+lag" #worse
            #train_years = 5
            #cols = atr_adx_cols + vola_cols + ma_num + ma_rel + ma_sma + volu_cols + vix_skew_cols
        elif r == 20:
            base_cols = atr_adx_cols + vola_cols + ma_num + ma_sma
            #base_cols = ma_lag + ma_rel + ma_sma + ma_num + rsi_cols + macd_cols + volu_cols + atr_adx_cols + vola_cols + vix_skew_cols + experimental_slope_cols
            list_name = "initial+volu" #worse
            list_name = "initial+lag" #much worse
            #train_years = 6
        else:
            base_cols = atr_adx_cols + vola_cols + ma_num + ma_sma + volu_cols + rsi_cols + macd_cols
            base_cols = ma_lag + ma_rel + ma_sma + ma_num + rsi_cols + macd_cols + volu_cols + atr_adx_cols + vola_cols + vix_skew_cols + experimental_slope_cols
            list_name = "initial+volu" #worse
            list_name = "initial-rsimacd+volu" #better
            #train_years = 5
            models = {"xgboost-2": XGBClassifier(n_estimators=200, random_state=42, n_jobs=-1)}

        target_col = f"Return_{r}"
        # Trime unknown (recent) outcomes
        df_final = df_main.iloc[r:].copy()

        for train_year in train_years:

            df_scores = walkback_runs(
                df=df_final,
                feature_cols=base_cols,
                target_col=target_col,
                date_col="Date",
                train_years=train_year,
                test_days=test_day,
                step_days=test_day,
                runs=runs,
                horizon_days=r,
                purge_days=r, 
                fill_inf=0.0,
            )

            df_scores["feature_set"] = "past_returns" #list_name
            df_scores["horizon"] = r

            results.append(df_scores)

results_df = pd.concat(results, ignore_index=True)
#results_df.to_csv("baseline.csv", index=False)

PI Train: 2022-02-16 → 2026-01-13
Ran permutation importance for horizon 2 | Len: 484 | Old: 14 | New: 6
['Past_Return%_1', 'Past_Return%_10', 'Past_Return%_2', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5']
Run 1/230 | Train: 2022-02-16 → 2026-01-13 | Test: 2026-01-16 → 2026-01-16 | Train_n=980 | Test_n=1 | (PI Years: 2 - Feats: 6)
PI Train: 2022-02-15 → 2026-01-12
Ran permutation importance for horizon 2 | Len: 484 | Old: 14 | New: 6
['Past_Return%_1', 'Past_Return%_10', 'Past_Return%_2', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5']
Run 2/230 | Train: 2022-02-15 → 2026-01-12 | Test: 2026-01-15 → 2026-01-15 | Train_n=980 | Test_n=1 | (PI Years: 2 - Feats: 6)
PI Train: 2022-02-14 → 2026-01-09
Ran permutation importance for horizon 2 | Len: 484 | Old: 14 | New: 6
['Past_Return%_1', 'Past_Return%_10', 'Past_Return%_2', 'Past_Return%_3', 'Past_Return%_30', 'Past_Return%_5']
Run 3/230 | Train: 2022-02-14 → 2026-01-09 | Test: 2026-01-14 → 2026-01-14 | Train_n=980 | Test_n=

KeyboardInterrupt: 

In [413]:
df = pd.read_csv("horizon2_best.csv")
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon',
        'pi_size', 'min_feats']
df_new = results_df[cols].copy()
df_concat = pd.concat([df[cols], df_new], ignore_index=True)
df_concat.to_csv('horizon2_new.csv', index=False)

In [356]:
results_df.columns

Index(['run', 'model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n',
       'test_neg_n', 'test_pos_frac', 'test_neg_frac', 'train_n',
       'train_start', 'train_end', 'test_start', 'test_end', 'train_years',
       'n_features', 'pi_size', 'min_feats', 'feature_set', 'horizon'],
      dtype='object')

In [323]:
df = pd.read_csv("best_models_perf.csv")
#df.to_csv('best_models_backup.csv', index=False)
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon']
df_new = results_df[cols].copy()
df_concat = pd.concat([df, df_new], ignore_index=True)
df_concat.to_csv('best_models_perf.csv', index=False)

In [289]:
df_concat.drop_duplicates()
df_concat.drop_duplicates().to_csv('best_models_perf.csv', index=False)

In [186]:
cols = ['model', 'test_days', 'pred', 'acc', 'test_n', 'test_pos_n', 'train_n', 'test_start', 'test_end', 'train_years', 'feature_set', 'horizon']
df_new = results_df[cols].copy()
df_new.to_csv('performance.csv', index=False)